In [1]:
lst = ["ca",
"co",
"ct",
"de",
"hi",
"il",
"md",
"ma",
"nj",
"nm",
"ny",
"or",
"ri",
"vt",
"va",
"wa",]

lst = [x.upper() for x in lst]
print(lst)

['CA', 'CO', 'CT', 'DE', 'HI', 'IL', 'MD', 'MA', 'NJ', 'NM', 'NY', 'OR', 'RI', 'VT', 'VA', 'WA']
